In [1]:
# import the necessary packages
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec
from webscraper import webscraper as ws
import requests
import os

load_dotenv()

/root/projects/ai-rmp/.venv/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


True

In [2]:
# initialize the APIs
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
global_val = 1

In [12]:
pc.create_index("rmp-index", dimension=384, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1"))

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '91a16626c5c00121d0a581a72da9c6b8', 'Date': 'Sun, 25 Aug 2024 01:00:05 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [3]:
def getEmbeddings(model_id: str, hf_token: str, data: list[str]):
    api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"
    headers = {"Authorization": f"Bearer {hf_token}"}

    response = requests.post(api_url, headers=headers, json={"inputs": data, "options":{"wait_for_model":True}})
    return response.json()

model_id = "sentence-transformers/all-MiniLM-L6-v2"
hf_token = os.getenv("HUGGINGFACE_API_KEY")

processed_data = []
for i in range(global_val, global_val + 10):
    rmp_url = f"https://www.ratemyprofessors.com/professor/{i}"
    data = ws.scrape_rmp_link(rmp_url)

    if (data['prof_name'] == "N/A"):
        print("Professor not found")
        continue

    embeddings = getEmbeddings(model_id, hf_token, data['prof_name'])

    processed_data.append(
        {
            "values": embeddings,
            "id": data['prof_name'],
            "metadata": {
                "comments": data['comments'],
                "department": data['prof_dept'],
                "university": data['university_name'],
                "rating": data['rating'],
                "top_tags": data['top_tags'],
                "difficulty": data['difficulty'],
                "classes_taught": data['classes_taught'],
            }
        }
    )

global_val += 10

# Insert the embeddings into the Pinecone index
index = pc.Index("rmp-index")
upsert_response = index.upsert(
    vectors=processed_data,
    namespace="professors",
)
print(f"Upserted count: {upsert_response['upserted_count']}")

# Print index statistics
print(index.describe_index_stats())




[Professor Info]

Overall rating - 2.6/5
Number of ratings - 38 ratings
Professor name - aaa aaa 
Professor department - Computer Science department
University name - San Jose State University
Level of difficulty - 4.6/5
Would take again - 100%
Classes taught - [' RMP', ' ARTS1301', ' CS149', ' CS151', ' CS1', ' CS146']
Top tags - ['Inspirational', 'Caring', 'Respected']
Recent comments - ['b', 'Is this a RMP easter egg? ', 'Awesome teacher who really cares about making sure that you understand the material. Take him!', 'Really really great cares a lot about your grades and helping. You make clases so interesting.  One of my favorites classes.', 'A woman gave you worst nightmare. Always writing the content of textbook in the blackboard but not teaching you. Never willing to answer question if you did not understand.', "This professor has a rotten soul. Tests are hard, not because of the content, but because she will take off major points for minor technicalities, correct answers that 

ListConversionException: Expected a list or list-like data structure, but got: {'error': 'Rate limit reached. You reached free usage limit (reset hourly). Please subscribe to a plan at https://huggingface.co/pricing to use the API at this rate'}